# Analysis of Titanic Dataset
Author: Daniel Pipkin

In [3]:
%matplotlib inline
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from matplotlib import style
style.use('seaborn-pastel')